### 네이버 뉴스 정보 가져오기

### 학습 내용 
 * 네이버 뉴스 여러개 단어 검색
 * 페이지별 언론사, 뉴스기사, URL

### 01. 웹 브라우저 띄우기

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

### 네이버 뉴스 검색

In [3]:
driver = webdriver.Chrome('chromedriver_91')

url = 'https://search.naver.com/search.naver?query=apple&where=news'
driver.get(url)

import time
time.sleep(3)  # 홈페이지 로딩 시간 3초

In [14]:
# keyword_multi = ['이강덕 포항시장 백신', '태백시 류태호 백신', '허석 순천시장 백신', 
#                  '박영선 백신', '이강덕 시장 백신', '고3 백신', '최대호 안양시장 백신']

keyword_multi = ['고3 백신부작용', '청소년백신부작용']
                 
pages = 30

In [15]:
### 뉴스 검색 : //*[@id="nx_query"]
### 검색 버튼 : //*[@id="lnb.searchForm"]/fieldset/button

In [16]:
from datetime import datetime

now = datetime.now()
file_make_time = "%04d_%02d%02d_%02d" % (now.year, now.month, now.day, now.hour)
print(now.day - 1)
now_day = now.day
now_hour = now.hour
file_make_time

10


'2021_0911_13'

In [17]:
import re
from datetime import timedelta

In [18]:
def date_cal(now_time):
    now = datetime.now()

    # 시간이 포함되어 있고,
    chk_time = now_time.find("시간")
    chk_day = now_time.find("일")
    if chk_time>=0: 
        hour = int( re.findall("\d+", now_time)[0] )
        
        # 현재 시간 - 적힌 시간이 음수가 나오면 현재 날짜에 -1을 해 준다. 
        cal_hour = now.hour - hour
        # print(cal_hour)
        if cal_hour < 0:
            news_date = timedelta(days = 1)
            cal_now = now - news_date
            #print("현재 날짜 :",  cal_now )
        else:
            news_date = timedelta(days = 0)
            cal_now = now - news_date
            #print("현재 날짜 :",  cal_now )
            
        cal_date_time = "%04d-%02d-%02d" % (cal_now.year, cal_now.month, cal_now.day)
        print("뉴스 날짜 : ", cal_date_time)
    
    # 일이 포함되어 있고,
    elif chk_day>=0: 
        news_day = int( re.findall("\d+", now_time)[0] )
        # print(news_day)
        
        # 현재 날짜 -1일전을 계산한다.
        news_date = timedelta(days = news_day)
        cal_now = now - news_date
        
        cal_date_time = "%04d-%02d-%02d" % (cal_now.year, cal_now.month, cal_now.day)
        # print("뉴스 날짜 : ", cal_date_time)
    else:
        cal_date_time = now_time
    return cal_date_time    

In [19]:
# date_cal("3일 전")

In [20]:
def keyword_news_n(keyword1, now_time):
    now_page_num = 0
    all_company1 = []
    all_article1 = []
    all_link1 = []
    all_date1 = []
    
    for i in range(1,pages,1):
        # 페이지 부분 정보 획득
        sel_num = driver.find_element_by_class_name("sc_page_inner")
        sel_all = sel_num.find_elements_by_class_name("btn")
        
        # 화면에 보이는 페이지 확인
        print("현재 페이지 : ", now_page_num+1, "키워드 : ", keyword1, "작성시간: ", now_time)
        len_all = len(sel_all)
        
        # 페이지 번호 확인 후, 해당되는 버튼 클릭
        for i in range(1, len_all+1 ):
            tmp_pre = '//*[@id="main_pack"]/div[2]/div/div/a[' + str(i) + ']'
            sel_num = driver.find_element_by_xpath(tmp_pre)
            sel_page_num = int(sel_num.text)
            if sel_page_num == (now_page_num+1):
                # 정보를 가져와서 클릭
                # 현재 페이지 입력
                sel_num.click()
                time.sleep(1)
                now_page_num += 1 
                print(now_page_num)
                break

        page = driver.page_source
        soup = BeautifulSoup(page, 'html.parser')
        all_ul = soup.find("ul", class_="list_news").find_all("li", class_="bx")

        for one in all_ul:
            # 언론사
            com = one.find("a", class_="info press").text
            com_a = com.replace("언론사 선정", "")

            # print(one)
            tmp = one.find("a", class_="news_tit")

            article_n = tmp.text
            tmp_link = tmp.attrs['href']
            
            # 날짜
            date_one = one.find('div', class_='info_group').find("span", class_="info").text
            now_dates = date_cal(date_one)  # 날짜 계산
            
            print(article_n, com_a, now_dates)
            print("url : ", tmp_link)

            all_company1.append(com_a)
            all_article1.append(article_n)
            all_link1.append(tmp_link)
            all_date1.append(now_dates)
            
        print(len(all_article))
    return all_company1, all_article1, all_link1, all_date1

In [21]:
import pandas as pd
from datetime import datetime
import os

### 폴더 생성

In [22]:
now = datetime.now()
file_make_time = "%04d_%02d%02d_%02d" % (now.year, now.month, now.day, now.hour)

try:
    if not os.path.exists(file_make_time):
        os.makedirs(file_make_time)
        print("folder creating ok")
except OSError:
    print("Error : Creating directory ", file_make_time)

folder creating ok


In [23]:
now = datetime.now()
file_make_time = "%04d_%02d%02d_%02d" % (now.year, now.month, now.day, now.hour)

all_company = []
all_article = []
all_link = []
all_date = []

for one in keyword_multi:
    url = 'https://search.naver.com/search.naver?query=' + one + '&where=news'
    driver.get(url)
    time.sleep(3)
    
    sel_search = driver.find_element_by_xpath('//*[@id="nx_query"]') 
    sel_btn = driver.find_element_by_xpath('//*[@id="nx_search_form"]/fieldset/button') 
    
    sel_search.clear()
    sel_search.send_keys(one)
    sel_btn.click()
    
    all_company, all_article, all_link, all_date = keyword_news_n(one, 
                                                                  file_make_time)
    
    dict_dat = {"언론사": all_company, "기사명" : all_article, 
                "날짜":all_date, "URL":all_link}
    
    dat = pd.DataFrame(dict_dat)
    dat['키워드'] = one
    dat.columns = ['키워드', "언론사", "기사명", "날짜", "URL"]
    
    fname_csv = file_make_time + "/" + one + "_" + file_make_time +".csv"
    fname_xlsx = file_make_time + "/" + one + "_" + file_make_time +".xlsx"
    dat.to_csv(fname_csv, index=False)
    dat.to_excel(fname_xlsx, index=False)

현재 페이지 :  1 키워드 :  고3 백신부작용 작성시간:  2021_0911_13
1
‘백신 티켓’ 사실이었나?… 모평 N수생 증가폭이 수능보다 컸다 조선일보 2021-09-06
url :  https://www.chosun.com/national/education/2021/09/06/CSPLSWENAVFGFGVNQHUY4DLMDM/?utm_source=naver&utm_medium=referral&utm_campaign=naver-news
중학생 우리 아이 백신 맞아도 될까? 오마이뉴스 2021.08.31.
url :  http://www.ohmynews.com/NWS_Web/View/at_pg.aspx?CNTN_CD=A0002770822&CMPT_CD=P0010&utm_source=naver&utm_medium=newsearch&utm_campaign=naver_news
백혈병 하혈 설사 통증…백신 부작용 불안감에 2030 '열공 모드' 매일경제 2021.09.02.
url :  http://news.mk.co.kr/newsRead.php?no=847715&year=2021
청년은 코로나보다 백신이 더 위험할까? 전문가에게 물었습니다 KBS 2021.08.26.
url :  https://news.kbs.co.kr/news/view.do?ncd=5265267&ref=A
[뉴스쉽] 보건소 "잘 몰라" 질병청 "통화 중"…백신 궁금하면 드루와 SBS 2021.08.28.
url :  https://news.sbs.co.kr/news/endPage.do?news_id=N1006445151&plink=ORI&cooper=NAVER
'코로나 백신 부작용' 교원 불안감 상승 금강일보 2021.09.02.
url :  http://www.ggilbo.com/news/articleView.html?idxno=866962
수능 때문에 코로나 백신 맞은 고3, 부작용 얼마나 겪었나? 헬스조선 2021.08.15.
url :  https://health.chosun.com

8
고3 백신 접종 시작‥고등학교 교직원도 접종 EBS 2021.07.19.
url :  http://news.ebs.co.kr/ebsnews/allView/60080094/N
의료인 연합, 고3 ‘화이자 백신’ 접종 무기한 연기 청원…청소년 백신 안전... 디지틀조선일보 2021.07.10.
url :  http://digitalchosun.dizzo.com/site/data/html_dir/2021/07/10/2021071080022.html
고3 백신 접종 시작.. '장거리 접종' 불편 호소 MBC충북 2021.07.19.
url :  http://www.mbccb.co.kr/rb/?r=home&c=73/79&mod=view&seq=0023&rvdate=20210719
[르포] 고3 백신 접종센터 가보니…"이제 마음 놓고 공부에 집중해야죠" 경북일보 2021.07.20.
url :  http://www.kyongbuk.co.kr/news/articleView.html?idxno=2079638
고3 백신 접종 첫날 긴장 속 차분하게 시행…유은혜 “고1·2 접종 협의 중” 이투데이 2021.07.19.
url :  https://www.etoday.co.kr/news/view/2045666
앳된 얼굴 고3 학생 백신 맞자 "마음 놓여"…미성년자 첫 예방접종 뉴스1 2021.07.19.
url :  https://www.news1.kr/articles/?4376206
[코로나19] 충북 고3 백신 접종 시작…"마음 편히 수능 준비" BBS NEWS 2021.07.19.
url :  https://news.bbsi.co.kr/news/articleView.html?idxno=3038800
“겁나서 못 맞겠다” 독감 백신 접종 절반으로 뚝 강원일보 2020.10.27.
url :  http://www.kwnews.co.kr/nview.asp?aid=220102600059
[영상 르포] 고3수험생 코로나19 첫 백신접종 "큰 변화 있을지 모르지만 마음... 뉴스클레

15
고3 백신, 30일까지 동의안하면 차례 밀린다…이틀간 출석인정 중앙일보 2021.06.28.
url :  https://news.joins.com/article/olink/23687494
"젊은층 먼저·지자체 우선" 코로나19 백신 우선순위 논란…임의적 분배 부작... 메디게이트뉴스 2021.07.07.
url :  http://www.medigatenews.com/news/3062294317
갑작스런 백신 접종 간격 연장…"원칙없는 변경 문제" 메디파나뉴스 2021.08.11.
url :  http://medipana.com/news/news_viewer.asp?NewsNum=282526&MainKind=A&NewsKind=5&vCount=12&vKind=1
학교발 코로나 감염 확산...고3 “백신 빨리 맞았으면...” 헤럴드경제 22면 1단
url :  http://news.heraldcorp.com/view.php?ud=20210602000620
4단계에도 2학기부터 초등 1·2학년, 고3 매일 등교한다 서울신문 4면 TOP
url :  https://www.seoul.co.kr/news/newsView.php?id=20210810004019&wlog_tag3=naver
mRNA 백신 부작용 심장 근육에 염증이... 한미약품 에스티팜 아이진 서린바이... 핀포인트뉴스 2021.07.02.
url :  http://www.pinpointnews.co.kr/view.php?ud=202107020652548210861beb2d9d_45
SK바이오 ‘코로나 백신’ 내년 출시 목표…‘비교임상’ 문제없나 쿠키뉴스 2021.08.10.
url :  http://www.kukinews.com/newsView/kuk202108100110
"감염 차단 위해" vs "아이들은 득보다 실" 고3 백신 접종 갑론을박 중앙일보 2021.06.03.
url :  https://news.joins.com/article/olink/23668544
백신 접종 간격 4→6주 조정… 

22
모더나 백신 공급 차질로 50대 접종 불안…심근염 사망 첫 인정 MBN 2021.07.26.
url :  http://www.mbn.co.kr/pages/news/newsView.php?news_seq_no=4559832
스마트학생복 설문조사, 2학기 전면 등교 "47.5%, 긍정적"..'백신 접종'은 70... 데일리경제 2021.07.21.
url :  http://www.kdpress.co.kr/news/articleView.html?idxno=104754
고3, 19일부터 화이자 순차 접종...50대는 26일부터 모더나 포쓰저널 2021.07.01.
url :  http://www.4th.kr/news/articleView.html?idxno=2001595
.고3 백신접종 8월 완료 계획..학부모 걱정 커 울산MBC 2021.05.18.
url :  https://usmbc.co.kr/article/AJm0uDLkQQL
“우리도 백신접종 해주세요”… 코로나19 18개월 민원만 44만건 중도일보 2021.07.20.
url :  http://www.joongdo.co.kr/web/view.php?key=20210720010004695
[뉴스라이브] 50대 백신 예약 3번째 '먹통'...청해부대 오늘 저녁 귀환 YTN 2021.07.20.
url :  https://www.ytn.co.kr/_ln/0103_202107201010417079
코로나19 백신, 여성건강에 영향?… “연구 드물어 추정 불가” 쿠키뉴스 2021.07.20.
url :  http://www.kukinews.com/newsView/kuk202107190239
[등교·백신 엇박] "원격수업이 편해…내 아이 백신 불안" 아주경제 2021.07.05.
url :  http://www.ajunews.com/view/20210704183930006
“50대 주로 맞을 모더나 백신, 1차만 맞아도 델타 72% 예방” 중앙일보 A3면 1단
url :  https://news.joins.com

29
하루 64만 명 접종 속도전…50대·고3은 7월 MBC 2021.05.28.
url :  https://imnews.imbc.com/replay/2021/nwtoday/article/6196998_34943.html
어제 하루 64만 명 접종…7월부터 50대 · 고3 수험생 SBS 2021.05.28.
url :  https://news.sbs.co.kr/news/endPage.do?news_id=N1006334591&plink=ORI&cooper=NAVER
정부 “화이자 백신, 고3 이어 16~17세 접종도 검토” 서울신문 2021.04.24.
url :  https://www.seoul.co.kr/news/newsView.php?id=20210424500062&wlog_tag3=naver
7월 50대 우선 백신접종... 49세 이하는 8월부터 조선일보 A1면 1단
url :  https://www.chosun.com/national/welfare-medical/2021/06/16/6HT7FKTOORHP5LQBARERTT23JA/?utm_source=naver&utm_medium=referral&utm_campaign=naver-news
백신, 대안 없는 정부… 부작용 논란에도 'AZ접종 재개'에 무게 한국일보 1면 1단
url :  https://hankookilbo.com/News/Read/A2021040815190005931?did=NA
다음 달 코로나19 백신 교사 접종‥고3 학생도 방학 중 시작 EBS 2021.03.18.
url :  http://news.ebs.co.kr/ebsnews/allView/20473950/N
AZ 대신 화이자로 교차접종, 효과는? | 고3은 화이자, 교직원은 화이자 또는 ... topclass 2021.06.17.
url :  http://topclass.chosun.com/daily/view.asp?idx=1166&Newsnumb=2021061166
전면등교 대비 교직원 '백신무장'…미신청·취소자 구멍 어쩌나 뉴

7
[코로나19 비상] 추석 전 백신 1차 접종 70% 달성 관건은 '젊은층' 매일일보 2021.08.24.
url :  http://www.m-i.kr/news/articleView.html?idxno=855079
'위드 코로나'로 전환하는 백신 선진국…한국은 언제쯤 UPI뉴스 2021.08.30.
url :  http://www.upinews.kr/newsView/upi202108300115
수능 때문에 코로나 백신 맞은 고3, 부작용 얼마나 겪었나? 헬스조선 2021.08.15.
url :  https://health.chosun.com/site/data/html_dir/2021/08/13/2021081301270.html
HPV백신 NIP '17세 이하 남아' 적용 필요성 대두 데일리팜 2021.08.28.
url :  http://www.dailypharm.com/Users/News/NewsView.html?ID=279734&REFERER=NP
[주목! e기술] 코로나19 백신 긴급사용승인과 정식승인 차이 이데일리 2021.08.14.
url :  http://www.edaily.co.kr/news/newspath.asp?newsid=01341526629147320
美 “모더나, 30세 이하 남성 심근염 부작용 가능성" 조사중 중앙일보 2021.08.20.
url :  https://news.joins.com/article/olink/23726934
18∼49세 백신 접종 현재 예약률 64% 목표치 미달 정부 고심 인천일보 2021.08.22.
url :  http://www.incheonilbo.com/news/articleView.html?idxno=1109474
EMA, ‘다형홍반 등 화이자·모더나 백신과 연관성 검토 세계일보 2021.08.12.
url :  http://www.segye.com/content/html/2021/08/12/20210812510287.html?OutUrl=naver
코로나19 백신 맞은 청소년 심근염·심낭염 발생?

14
화이자 코로나19 백신, 만 12~15세 청소년에서 100% 예방률 메디게이트뉴스 2021.05.29.
url :  http://www.medigatenews.com/news/1682439380
이스라엘 보건부 "어린이·청소년에 화이자 백신 접종···심각한 부작용 없... 서울경제 2021.03.11.
url :  https://www.sedaily.com/NewsView/22JRLO7699
<글로벌 pick> 英 코로나 재확산..16세 미만 청소년 접종 '희망' EBS 2021.07.06.
url :  http://news.ebs.co.kr/ebsnews/allView/60074014/N
美 12∼15세, 화이자 백신 맞는다… 국내 청소년에도 접종 길 열리나 서울신문 5면 1단
url :  https://www.seoul.co.kr/news/newsView.php?id=20210512005018&wlog_tag3=naver
"환절기 감기와 비슷…48시간 내 사망할 수도" 의사들 경고 [이선아 기자의 생... 한국경제 A17면 1단
url :  https://www.hankyung.com/society/article/2021091075861
[팩트체크] 코로나는 감기? 서울대 명예교수의 백신 허위정보 뉴스톱 2021.08.18.
url :  https://www.newstof.com/news/articleView.html?idxno=12098
화이자 백신 30세 이하 남성 심근염 연관성 제기 팜뉴스 2021.06.03.
url :  http://www.pharmnews.com/news/articleView.html?idxno=104890
화이자·모더나 백신 부작용으로 심근염·심낭염 호소하는 환자들, 어떻게 진... 메디게이트뉴스 2021.08.12.
url :  http://www.medigatenews.com/news/1493640146
소청과 의사회 “AZ백신 연장자에게 접종해야” 스카이데일리 2021.03.18.
url :  http://www.sk

21
日 신규 확진자 만4천여 명 '최다'...英 16~17세 곧 백신 접종 YTN 2021.08.05.
url :  https://www.ytn.co.kr/_ln/0104_202108050843286198
영국 16∼17세 코로나19 백신접종 곧 시작…"연령 확대 검토" 연합뉴스TV 2021.08.05.
url :  http://www.yonhapnewstv.co.kr/MYH20210805003100038/?did=1825m
[분석] '화이자 부작용' 심근염은 무엇인가? 뉴스톱 2021.07.28.
url :  https://www.newstof.com/news/articleView.html?idxno=12056
모더나 코로나 백신 EU 10代 연령대 승인지지 약업신문 2021.07.26.
url :  http://www.yakup.com/news/index.html?mode=view&cat=16&nid=259458
시노백 백신 2번 맞은 인니 남성, 코로나19 확진 후 사망 아시아타임즈 2021.07.31.
url :  https://www.asiatime.co.kr/article/20210731500039
모더나 코로나 백신 스위스서 청소년用 허가신청 약업신문 2021.06.15.
url :  http://www.yakup.com/news/index.html?mode=view&cat=16&nid=258242
[코로나19] 백신접종 보릿고개…“우리 아이들은 언제 맞을 수 있나” 아주경제 2021.07.29.
url :  http://www.ajunews.com/view/20210729132752082
NYT “적당한 음주로 백신 면역반응 손상안돼".. 남성 하루 2잔 이하·여성 1... 세계일보 2021.07.30.
url :  http://www.segye.com/content/html/2021/07/30/20210730511949.html?OutUrl=naver
18~49세, 내달 26일 접종 시작…집단면역 성공 백신 수급에 달려 뉴스1 2021.07.

28
기대·우려 교차속 '고3 백신접종' 시작 전국매일신문 2021.07.19.
url :  http://www.jeonmae.co.kr/news/articleView.html?idxno=848191
백신 조롱곡 부르던 美방송인, 코로나 걸리자 "백신 맞아라" 중앙일보 2021.07.26.
url :  https://news.joins.com/article/olink/23708239
브라질, 신규 확진 10만 명대로 급증...유럽의약품청, 모더나 청소년 접종 권... YTN 2021.07.24.
url :  https://www.ytn.co.kr/_ln/0104_202107241206045256
교통사고엔 무기징역, 전염병 백신은 차질 “한심한 이율배반” 스카이데일리 2021.07.26.
url :  http://www.skyedaily.com/news/news_view.html?ID=136707
"안전하게 수능 봐야죠" 오늘부터 고3 백신 접종 시작 중앙일보 2021.07.19.
url :  https://news.joins.com/article/olink/23703258
‘수능까지 건강히’ 고3 코로나 백신 접종 시작![포착] 국민일보 2021.07.19.
url :  http://news.kmib.co.kr/article/view.asp?arcid=0016070198&code=61121111&cp=nv
청소년 절반 "2학기 전면 등교 긍정적" 데이터솜 2021.07.30.
url :  http://www.datasom.co.kr/news/articleView.html?idxno=116633
앳된 얼굴 고3 학생 백신 맞자 "마음 놓여"…미성년자 첫 예방접종 뉴스1 2021.07.19.
url :  https://www.news1.kr/articles/?4376206
'먹는 백신' 최초 임상 추진…코로나 '게임 체인저' 되나? SBS 2021.07.24.
url :  https://news.sbs.co.kr/news/endPage.do?news_id=N10